In [3]:
import torch

print("GPU 사용 가능 여부:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU 이름:", torch.cuda.get_device_name(0))

GPU 사용 가능 여부: False


In [ ]:
import pandas as pd
import multiprocessing as mp
from transformers import pipeline
from tqdm import tqdm
import os

# ✅ Step 1: 데이터 불러오기
CSV_PATH = "comments.csv"  # 여기에 파일명 넣기
COMMENT_COLUMN = "comment"  # 댓글이 담긴 컬럼명 (예: "내용", "text")

df = pd.read_csv(CSV_PATH)
texts = df[COMMENT_COLUMN].dropna().tolist()

# ✅ Step 2: 감정 레이블 정의
labels = ["긍정", "부정", "분노", "불안", "기대", "중립"]

# ✅ Step 3: 워커 함수 (프로세스마다 실행됨)
def analyze_sentiment(text):
    try:
        local_classifier = pipeline("zero-shot-classification", 
                                    model="joeddav/xlm-roberta-large-xnli",
                                    device=-1  # ← GPU 대신 CPU 사용)
                                    )
        result = local_classifier(text, labels)
        return {
            'text': text,
            'top_emotion': result['labels'][0],
            **{label: result['scores'][i] for i, label in enumerate(result['labels'])}
        }
    except Exception as e:
        return {
            'text': text,
            'top_emotion': 'error',
            **{label: None for label in labels}
        }

# ✅ Step 4: 병렬 실행
if __name__ == "__main__":
    print("🧠 프로세스 수:", mp.cpu_count())
    with mp.Pool(processes=mp.cpu_count() - 1) as pool:
        results = list(tqdm(pool.imap(analyze_sentiment, texts), total=len(texts)))

    # ✅ Step 5: 결과 저장
    result_df = pd.DataFrame(results)
    result_df.to_csv("감정분석_결과.csv", index=False)
    print("✅ 감정분석 완료 및 결과 저장됨!")